# **1. Shallow Learning**

In [ ]:
import numpy as np
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import numpy as np
import pandas as pd
import torch
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from gensim.models import Word2Vec
from tensorflow.keras.utils import to_categorical

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Cargamos el  dataset tokenizado
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")

# Revisamos la columna de etiquetas
print("Clases en 'bias':", df_train["bias"].value_counts())

# Creamos la  carpeta para modelos si no existe
os.makedirs("data/models", exist_ok=True)

# Calculamos TF-IDF sobre df_train 
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)
)
X_tfidf = tfidf_vectorizer.fit_transform(df_train["tfidf_joined"])
y = df_train["bias"].values

# Separamos el  train/validation 
X_tr, X_val, y_tr, y_val = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42
)

#  Definimos los 4 modelos que vamos a utilizar
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM (Linear)": SVC(kernel='linear'),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=200, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# Entrenamos y evaluamos los modelos
results = {}

for name, model in models.items():
    print(f"Entrenando {name}...")
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro')
    results[name] = {"Accuracy": acc, "Macro-F1": f1}
    # Guardar modelo
    pickle.dump(model, open(f"data/models/{name.replace(' ', '_').lower()}.pkl", "wb"))

#  Guardamos el  vectorizador
pickle.dump(tfidf_vectorizer, open("data/features/tfidf_vectorizer.pkl", "wb"))

# Resultados
results_df = pd.DataFrame(results).T
print("\nResultados comparativos de Shallow Learning:")
print(results_df)



c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10


Para evaluar los modelos hemos usado Accuracy y Macro-F1, métricas adecuadas para problemas de clasificación con clases desbalanceadas.
Los cuatro modelos presentan resultados parecidos. Sin embargo, XGBoost ofrece el mejor rendimiento, mostrando que puede capturar patrones complejos del sesgo ideológico mejor que modelos lineales o Random Forest.
Por otro lado, los modelos lineales funcionan razonablemente bien, lo que indica que el sesgo tiene señales lineales claras en los términos más frecuentes.
Elegimos estos cuatro modelos (Logistic Regression, SVM, Random Forest y XGBoost) para cubrir tanto enfoques lineales como basados en árboles, y utilizar el dataset tokenizado con TF-IDF para representar el texto en forma dispersa, adecuada para Shallow Learning.

# **3. Modelos Deep**


En esta parte nos vamos a enfocar en la clasificación del sesgo. Los motivos son los siguientes:

1- Detectar la orientación política de una noticia es la tarea principal del proyecto. Además, esta tarea permite evaluar cómo los modelos y embeddings capturan matices semánticos y patrones discursivos.
2- Una vez concluida la clasificación de sesgo, se puede reutilizar la pipeline para las demás tareas de clasificación (medio y temática).
3- La columna bias presenta un número moderadamente equilibrado de ejemplos por clase, lo que permite ajustar la arquitectura y los hiperparámetros de manera controlada antes de afrontar tareas más complejas.

Las razones de haber elegido las combinaciones de embeddings y arquitecturas de redes neuronales para abordar la clasificación del sesgo ideológicoson las siguientes:

1- Comparación de embeddings no contextuales y contextuales:
    -Los embeddings no contextuales como, Word2Vecy y FastText, permiten capturar relaciones semánticas entre palabras de manera estática.
    -Los embeddings contextuales como, Sentence Transformers y BERT, capturan el significado de las palabras según su contexto en la frase, lo que es clave para detectar matices ideológicos más complejos.

2- Exploración de diferentes estrategias de embeddings:
    -Word2Vec congelado: usar embeddings preentrenados sin actualizar durante el entrenamiento, para evaluar la capacidad de vectores fijos.
    -Word2Vec fine-tune: ajustar los vectores durante el entrenamiento para adaptarlos al corpus específico.
    -Word2Vec “from scratch”: entrenar desde cero sobre el dataset, para capturar patrones propios del corpus.
    -Para los embeddings contextuales, se compara Sentence Transformers preentrenado frente a BERT, con fine-tuning parcial o total según la arquitectura de la red.

3- Elección de arquitecturas de redes neuronales:
    -Redes totalmente conectadas (Dense): adecuadas para embeddings agregados o promedio de vectores de palabras.
    -Redes recurrentes (LSTM/GRU): capturan secuencias y dependencias entre palabras, esenciales para comprender el flujo discursivo en los artículos.
    -CNN para texto: permiten identificar patrones locales de n-gramas que son relevantes en la clasificación de sesgo.

4- Razonamiento general:
    -Combinar diferentes tipos de embeddings y arquitecturas permite evaluar cuál representa mejor la información semántica y estilística para cada tarea.
    -Esta estrategia también permite analizar cómo el fine-tuning de embeddings impacta en la capacidad del modelo de captar señales ideológicas, frente a vectores preentrenados fijos.

Preparacion de los embeddings

In [ ]:



# ======================================================
# 1. Cargar dataset tokenizado
# ======================================================
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")

X_text = df["tokens"].apply(lambda x: " ".join(x))
y = df["bias"].values

# Split
X_train_text, X_val_text, y_train, y_val = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)


# ======================================================
# 2. Tokenizer → convertir textos a índices
# ======================================================
MAX_LEN = 200
VOCAB_SIZE = 20000  # suficiente para tu dataset

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train_text)

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=MAX_LEN)
X_val_seq   = pad_sequences(tokenizer.texts_to_sequences(X_val_text),   maxlen=MAX_LEN)


# ======================================================
# 3. Cargar Word2Vec entrenado anteriormente
# ======================================================
w2v = Word2Vec.load("data/embeddings/word2vec.model")
EMBEDDING_DIM = w2v.vector_size

# Construir matriz embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
for word, idx in tokenizer.word_index.items():
    if idx < VOCAB_SIZE and word in w2v.wv:
        embedding_matrix[idx] = w2v.wv[word]


# ======================================================
# 4. Función generadora de modelos LSTM
# ======================================================
def build_model(embedding_matrix=None, trainable=False, scratch=False):
    
    inputs = Input(shape=(MAX_LEN,))
    
    if scratch:
        embedding = Embedding(
            input_dim=VOCAB_SIZE,
            output_dim=EMBEDDING_DIM,
            input_length=MAX_LEN,
            trainable=True
        )(inputs)
    else:
        embedding = Embedding(
            input_dim=VOCAB_SIZE,
            output_dim=EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=MAX_LEN,
            trainable=trainable
        )(inputs)

    x = LSTM(128, return_sequences=False)(embedding)
    x = Dropout(0.3)(x)
    outputs = Dense(3, activation="softmax")(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    return model


# ======================================================
# 5. Entrenar los tres modelos
# ======================================================
results = {}

# --- A) Word2Vec congelado ---
model_frozen = build_model(embedding_matrix, trainable=False)
history_frozen = model_frozen.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=5, batch_size=64, verbose=1
)

# --- B) Word2Vec fine-tuned ---
model_ft = build_model(embedding_matrix, trainable=True)
history_ft = model_ft.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=5, batch_size=64, verbose=1
)

# --- C) Embedding desde cero ---
model_scratch = build_model(scratch=True)
history_scratch = model_scratch.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=5, batch_size=64, verbose=1
)


# ======================================================
# 6. Evaluación
# ======================================================
def evaluate_model(model, name):
    y_pred = np.argmax(model.predict(X_val_seq), axis=1)
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average="macro")
    results[name] = (acc, f1)


evaluate_model(model_frozen,  "W2V Frozen")
evaluate_model(model_ft,      "W2V Fine-tuned")
evaluate_model(model_scratch, "Embedding Scratch")

df_results = pd.DataFrame(results, index=["Accuracy", "Macro-F1"]).T
print("\nResultados comparativos:\n")
print(df_results)


In [13]:
# Entrenar Word2Vec desde cero (o cargar modelo preentrenado)
w2v_model = Word2Vec(sentences=X_tr_text.tolist(),
                     vector_size=embedding_dim,
                     window=5,
                     min_count=3,
                     workers=4,
                     sg=1,
                     epochs=30)

# Función para convertir tokens a secuencias de vectores promedio
def tokens_to_w2v_avg(tokens, model, dim=100):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    if len(vecs) == 0:
        return np.zeros(dim)
    return np.mean(vecs, axis=0)

# Convertir train y validation
X_tr_w2v = np.array([tokens_to_w2v_avg(t, w2v_model, embedding_dim) for t in X_tr_text])
X_val_w2v = np.array([tokens_to_w2v_avg(t, w2v_model, embedding_dim) for t in X_val_text])


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_fl

In [14]:
# Cargar modelo preentrenado
st_model = SentenceTransformer("all-MiniLM-L6-v2")

# Convertir tokens a frases
X_tr_sent = [" ".join(t) for t in X_tr_text]
X_val_sent = [" ".join(t) for t in X_val_text]

# Obtener embeddings
X_tr_st = st_model.encode(X_tr_sent, batch_size=32, show_progress_bar=True)
X_val_st = st_model.encode(X_val_sent, batch_size=32, show_progress_bar=True)


Batches:   0%|          | 0/700 [00:00<?, ?it/s]

Batches:   0%|          | 0/175 [00:00<?, ?it/s]

In [15]:
from tensorflow.keras.utils import to_categorical

num_classes = len(np.unique(y))
y_tr_cat = to_categorical(y_tr, num_classes)
y_val_cat = to_categorical(y_val, num_classes)

def build_dense_model(input_dim, num_classes):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Dense + Word2Vec
dense_w2v = build_dense_model(embedding_dim, num_classes)
dense_w2v.fit(X_tr_w2v, y_tr_cat, epochs=10, batch_size=64, validation_data=(X_val_w2v, y_val_cat))

# Dense + Sentence Transformers
dense_st = build_dense_model(X_tr_st.shape[1], num_classes)
dense_st.fit(X_tr_st, y_tr_cat, epochs=10, batch_size=64, validation_data=(X_val_st, y_val_cat))


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.4111 - loss: 1.0687 - val_accuracy: 0.4736 - val_loss: 1.0297
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.4823 - loss: 1.0192 - val_accuracy: 0.5189 - val_loss: 0.9860
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5058 - loss: 0.9941 - val_accuracy: 0.5316 - val_loss: 0.9685
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.5229 - loss: 0.9756 - val_accuracy: 0.5447 - val_loss: 0.9570
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5293 - loss: 0.9667 - val_accuracy: 0.5481 - val_loss: 0.9511
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5386 - loss: 0.9546 - val_accuracy: 0.5534 - val_loss: 0.9381
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5405 - loss: 0.9522 - val_accuracy: 0.5486 - val_loss: 0.9417
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5474 - loss: 0.9432 - val_accura

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Convertir tokens a índices
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_tr_text)
X_tr_seq = tokenizer.texts_to_sequences(X_tr_text)
X_val_seq = tokenizer.texts_to_sequences(X_val_text)

# Padding
X_tr_seq = pad_sequences(X_tr_seq, maxlen=max_seq_len)
X_val_seq = pad_sequences(X_val_seq, maxlen=max_seq_len)

vocab_size = len(tokenizer.word_index) + 1

# LSTM con embedding inicializado aleatoriamente
def build_lstm_model(vocab_size, embed_dim, seq_len, num_classes):
    model = Sequential()
    model.add(torch.nn.Embedding(vocab_size, embed_dim, input_length=seq_len))
    model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [17]:
def evaluate_model(model, X_val, y_val_cat):
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_val_cat, axis=1)
    acc = accuracy_score(y_true_classes, y_pred_classes)
    f1 = f1_score(y_true_classes, y_pred_classes, average='macro')
    return acc, f1

acc_dense_w2v, f1_dense_w2v = evaluate_model(dense_w2v, X_val_w2v, y_val_cat)
acc_dense_st, f1_dense_st = evaluate_model(dense_st, X_val_st, y_val_cat)

print("Dense + Word2Vec: Accuracy =", acc_dense_w2v, "Macro-F1 =", f1_dense_w2v)
print("Dense + Sentence Transformers: Accuracy =", acc_dense_st, "Macro-F1 =", f1_dense_st)


175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Dense + Word2Vec: Accuracy = 0.5684417441029307 Macro-F1 = 0.5642255637362837
Dense + Sentence Transformers: Accuracy = 0.5591493924231594 Macro-F1 = 0.5544943496473945


# **4. Transformers**

# **5. Tabla Comaprativa de Resultados**

# **6. Interpretabilidad**